In [ ]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
def data_process():
    data_df = pd.read_csv('data.csv',usecols= ['countryEnglishName','provinceEnglishName','province_curedCount',
                                'province_deadCount','province_confirmedCount','updateTime'],encoding = 'utf-8')
    location_df = pd.read_csv('location.csv',usecols=['provinceEnglishName','longitude','latitude'])
    data_df['updateTime'] = data_df['updateTime'].apply(lambda x:x.split(' ')[0])
    data_df = data_df[data_df['updateTime']=='2020-04-14'].drop_duplicates(subset=['provinceEnglishName'], 
                                                                           keep='first')
    data_df=data_df[~((data_df['countryEnglishName']=='China') & (data_df['provinceEnglishName']!='China'))]
    df3 = pd.merge(data_df,location_df,on='provinceEnglishName',how='left')
    df3.dropna(axis=0, how='any',inplace=True)
    return df3
class Virus_Map:
    def __init__(self,df):
        self.df=df
        self.cid=None
        self.count = 0
        self.colors = ['r','g','gray']
        self.cols=['province_confirmedCount','province_curedCount','province_deadCount']
        self.titles=['全球累计确诊人数:','全球累计治愈人数:','全球累计死亡人数:']
        self.route(None)
    def draw(self):
        fig.canvas.mpl_disconnect(self.cid)
        m=Basemap()
        m.drawcoastlines()
        m.drawcountries(linewidth=1.5)
        m.drawmapboundary(fill_color ='#c5e6f9')
        plt.scatter(self.df['longitude'], self.df['latitude'], marker = 'o', color =self.colors[self.count%3],
                                                s=np.sqrt(self.df[self.cols[self.count%3]])*2.5, alpha=0.7)
        for i in range(len(self.df)):
            s=self.df.iloc[i]
            plt.annotate(s[self.cols[self.count%3]], xy = (s['longitude'],s['latitude']), 
                         xytext = (s['longitude'],s['latitude']),color='#262931')
        plt.text(s=self.titles[self.count%3],x=-160,y=-40,fontsize=20,fontweight='heavy',
                                                                            color=self.colors[self.count%3])
        plt.text(s=str(self.df[self.cols[self.count%3]].sum()),x=-160,y=-47,fontsize=20,
                                                                            color=self.colors[self.count%3])
        self.cid=fig.canvas.mpl_connect('button_press_event', lambda event:self.route(event))
        fig.canvas.draw()
        #plt.savefig('C:/Users/Lenovo/Desktop/'+str(self.count))
    def route(self,event):
        plt.cla()
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
        self.draw()
        self.count+=1
fig= plt.figure(figsize=(20,10))
ax=plt.axes([0.01,0.01,0.98,0.98])
virus_map=Virus_Map(data_process())
plt.show()